## Deep Metrics Learning Notebook.

In [1]:
# Imports

import tensorflow as tf
import keras
import keras.backend as K
from keras.callbacks import Callback
from keras.optimizers import Adam
import mlflow
import mlflow.keras
import numpy as np
import random

from scripts.data import prepare_pose, read_data, split_data_metrics_learning, get_req_ids, train_test, subsample
from scripts.model_config import base_model, offline_triplet_network, online_triplet_network
from scripts.losses import offline_triplet_loss
from tensorflow_addons.losses.triplet import triplet_semihard_loss
from scripts.offline_triplet_generator import offline_triplet_generator
from scripts.train import train
from scripts.evaluate import generate_embedding, build_annoy_index
from scripts.metrics import get_metrics
from scripts.model_eval_on_epoch_end import EvalOnEpochEnd
from scripts.mlflow_logger import MLFlowLogger

In [2]:
# Read data.h5 and preprocess the poses.
extracted_poses, transformed_poses, target = read_data(path="./data/data_v2.h5")
person_ids = target[:,0]
seat_ids = target[:,1]
actual_pose = prepare_pose(transformed_poses, seat_ids)

Set 1 for seat ids 2|3 and 0 for seat ids 0|1 ::
Camera perpective shape for each sequence::  (978,)
-----------------------------------------------------------------
Remove camera perpective dim from (978, 2, 120, 75)
Actual Pose shape after removing perspective dim::  (978, 120, 75)
-----------------------------------------------------------------
Reshape the last dim (pose - 75) into (x,y,score - 25 x 3)
Actual Pose shape ::  (978, 120, 25, 3)
-----------------------------------------------------------------
Eliminate score from (x,y,score) - 25 x 3 to get pose coordinates or 25-Joints(x,y) - 25 x 2
Actual Pose shape ::  (978, 120, 25, 2)
-----------------------------------------------------------------
Consider only 11 Joints - (7, 8, 9, 10, 11, 12, 13, 18, 20, 21, 23) out of 25 joints
Pick only the required joints from the actual pose which has all the 25 joints.
-----------------------------------------------------------------
Reshape the joints from 2d (11 x 2) to 1d (22,)
FINAL

In [3]:
# Set configurations.
cfg = {
    #data config
    'actual_pose': actual_pose,
    'target': target,
    'person_ids': person_ids,
    'val_ids': [1, 3, 4, 19, 20], # The ids mentioned will be the validation ids and the rest is taken for training.
    'num_of_val_ids': 5, # No. of val ishape per experiment. This is equivalent to the number of users using the car.
    'window_width' : 90,
    'overlap': 0.5,
    'random_state': 32, # For picking random sequences for anchor and gallery set.
    'random_seed': 45, # For picking random validation ids for the experiment.
    'augment_data': True, # or False -- Setting this False will avoid data augmentation.
    'anchor_gallery_split_size': 0.2, # % of anchor gallery split. Ex. 0.2 => 20% for anchor set and the rest for gallery set.
    'num_of_joints': 11,
        
    #triplet mining
    'mining': 'offline', # or 'online' -- Setting which mining strategy should be used. It could be online or offline.
    
    #model config
    'lstm_dropout': 0.8, # Dropout in LSTM units for offline model.
    '1d_spatial_dropout': 0.2, # Dropout in 1D Convolution block for offline model.
    'normal_dropout': 0.8, # Dropout for online mining model. 
    'vec_dim': 128, # No. of dimensions of the feature vector that represents the gait sequence.
    'epochs': 2,
    'batch_size': 64,
    'learning_rate': 0.001, # Learning rate of Adam optimizer.
    'ckpt_dir' : './models/', # Directory to which the model is saved after each epoch.
    'eval_interval': 3, # Interval in which the model is evaluated while training. Ex. setting this to 1 => evaluate model performance once for every 3 epochs.
    
    #experiment config
    'n_times_train': 10, # If this value is set to 10, then 10 different set of validation ids are selected and therefore 10 different experiments will run.
    
    #model path for inference.
    'model_path_inference': './models/model-val-[9, 14, 16, 20, 3]-90-128-offline.h5'
}

## Model Training

In [9]:
random.seed(cfg['random_seed'])
for _ in range(cfg['n_times_train']):
    val_ids = random.sample(range(1, 21), cfg['num_of_val_ids'])
    cfg['val_ids'] = val_ids
    train(cfg)

/home/aev32/anaconda3/envs/keras_env/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Epoch 1/2
22/22 [==============================] - ETA: 0s - loss: 0.2260	 Train Metrics - epoch: 1 - rank1_acc: 0.380000 - mAP: 0.489092
	 Val Metrics - epoch: 1 - rank1_acc: 0.540000 - mAP: 0.538710
22/22 [==============================] - 20s 888ms/step - loss: 0.2260 - val_loss: 0.1886
Epoch 2/2
22/22 [==============================] - ETA: 0s - loss: 0.2106	 Train Metrics - epoch: 2 - rank1_acc: 0.380000 - mAP: 0.481740
	 Val Metrics - epoch: 2 - rank1_acc: 0.500000 - mAP: 0.509027
22/22 [==============================] - 18s 821ms/step - loss: 0.2106 - val_loss: 0.1990


/home/aev32/anaconda3/envs/keras_env/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Epoch 1/2
23/23 [==============================] - ETA: 0s - loss: 0.2235	 Train Metrics - epoch: 1 - rank1_acc: 0.420000 - mAP: 0.552817
	 Val Metrics - epoch: 1 - rank1_acc: 0.375000 - mAP: 0.463690
23/23 [==============================] - 21s 900ms/step - loss: 0.2235 - val_loss: 0.2105
Epoch 2/2
23/23 [==============================] - ETA: 0s - loss: 0.2099	 Train Metrics - epoch: 2 - rank1_acc: 0.500000 - mAP: 0.571961
	 Val Metrics - epoch: 2 - rank1_acc: 0.458333 - mAP: 0.526324
23/23 [==============================] - 19s 806ms/step - loss: 0.2099 - val_loss: 0.2064


## Model Inference

In [4]:
# Load Model
model = keras.models.load_model(cfg['model_path_inference'], custom_objects={'offline_triplet_loss': offline_triplet_loss, 'tf':tf})

# Obtain anchor and gallery set.
X_set, y_set = get_req_ids(cfg['actual_pose'], cfg['target'], cfg['val_ids'], cfg['person_ids'])
X_gal, X_anchor, y_gal, y_anchor = train_test(X_set, y_set, test_size=cfg['anchor_gallery_split_size'], random_state=cfg['random_state'], stratify=y_set)
X_gal, y_gal = subsample(cfg=cfg, poses=X_gal, targets=y_gal, window_width = cfg['window_width'], overlap = cfg['overlap'])

# Generate feature vectors for the gallery set and generate the embedding space.
embedding_dict = generate_embedding(cfg, model, X_gal)

# Use Annoy library for indexing the feature vector so that k-Nearest neighbor can be retrived.
annoy_index = build_annoy_index(cfg, embedding_dict=embedding_dict)

# Print all evaluation metrics, rank-1, rank-5, rank-10 accuracy, mean Average Precision, most voted rank-10 accuracy and average correct resutls(accuracy).
rank1_acc , mAP = get_metrics(cfg, model, X_anchor, y_anchor, X_gal, y_gal, annoy_index, cfg['vec_dim'])

print('Rank1 Accuracy ', rank1_acc, 'mean Average Precision ', mAP)

/home/aev32/anaconda3/envs/keras_env/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
/home/aev32/anaconda3/envs/keras_env/lib/python3.7/site-packages/tensorflow/python/keras/layers/recurrent.py:820: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if (isinstance(inputs, collections.Sequence)
/home/aev32/anaconda3/envs/keras_env/lib/python3.7/site-packages/tensorflow/python/framework/tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Avg acc is :: 0.28
Rank 10 acc is :: 1.0
Rank 5 acc is :: 0.92
Rank 1 acc is :: 0.46
Mean Avg Precision is :: 0.5496943121693121
Vote res ::  0.46
Rank1 Accuracy  0.46 mean Average Precision  0.5496943121693121
